# Constructing TensorFlow Recommenders Model

In [1]:
import tensorflow_recommenders as tfrs

2023-06-13 20:07:50.028658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import numpy as np

In [3]:
import string

In [4]:
fiverr_df = pd.read_csv('/Users/rsoedarnadi/Documents/GitHub/machine-learning/fiverr_data.csv', delimiter=',')

In [5]:
fiverr_df.head(6)

,illustrator_name,gig_rating,gig_price_from,gig_img_url,gig_url,gig_design_type,gig_design_style,client_origins,client_names,illustrator_origins
0,nadia_younus,5.0,60,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/nadia_younus/do-minimal...,logos,minimalist,Canada,stoxventures,Pakistan
1,mariya_logo99,4.9,15,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mariya_logo99/do-creati...,logos,minimalist,Italy,lifehossain,Bangladesh
2,mountdesign,4.9,10,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mountdesign/design-3-pr...,logos,minimalist,United States,stormproofrooof,India
3,ahmed_gfx786,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/ahmed_gfx786/design-a-u...,logos,minimalist,France,remi_bgd,Pakistan
4,websutra,4.9,400,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/websutra/draw-an-unique...,logos,minimalist,United States,tribolumen,Sri Lanka
5,kiara43,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/kiara43/create-modern-m...,logos,minimalist,Germany,sustainifytoday,Sri Lanka


In [6]:
fiverr_df = fiverr_df.dropna().drop_duplicates().reset_index(drop=True)

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
price = fiverr_df[['gig_price_from']]

In [9]:
scaler = MinMaxScaler()

In [10]:
price=scaler.fit_transform(price)

In [11]:
fiverr_df['gig_price_from']=price

In [12]:
fiverr_df['gig_rating']=np.asarray(fiverr_df['gig_rating'],dtype=np.float64) / 5

In [13]:
cos_sim_df = fiverr_df[['gig_img_url','gig_price_from','gig_design_type','gig_design_style', 'illustrator_origins']]

In [14]:
def remove_punc(text):
    cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
    return cleaned

In [15]:
cos_sim_df['illustrator_origins']=cos_sim_df['illustrator_origins'].apply(remove_punc)

/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_6974/2612384505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['illustrator_origins']=cos_sim_df['illustrator_origins'].apply(remove_punc)


In [16]:
cos_sim_df['joined_feat'] = ''
cos_sim_df['joined_feat'] = cos_sim_df[cos_sim_df.columns[2:]].apply(lambda x: ' '.join(x), axis=1)

/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_6974/2216811935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['joined_feat'] = ''
/var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/ipykernel_6974/2216811935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cos_sim_df['joined_feat'] = cos_sim_df[cos_sim_df.columns[2:]].apply(lambda x: ' '.join(x), axis=1)


In [17]:
cos_sim_df.set_index('gig_img_url', inplace=True)

In [18]:
cos_sim_df=cos_sim_df[['gig_price_from', 'joined_feat']]

In [19]:
cos_sim_df.head()

,gig_price_from,joined_feat
gig_img_url,,
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/192437271/original/3fb8af5466dc09f13e2afaccf22c6708f0a8976e.jpg",0.018425,logos minimalist pakistan
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/292635463/original/e6d48abe79bee1a9cdc2c71e83c88513440fc393.jpg",0.003350,logos minimalist bangladesh
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/109611151/original/42d4814c676b9f4320152d541bb6c8be7cd83d39.jpg",0.001675,logos minimalist india
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/132488260/original/955d6d66789bd57075f25da7de2b3a4233b2e26a.jpg",0.000000,logos minimalist pakistan
"https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/122531603/original/eccdc57ef960b6cc51853d37a1ff9034af618deb.jpg",0.132328,logos minimalist sri lanka


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(cos_sim_df['joined_feat'])
tfidf_matrix.shape

(1686, 132)

In [22]:
helpp = tfidf_matrix.toarray()

In [23]:
helpp.shape

(1686, 132)

In [24]:
helppp = np.hstack((cos_sim_df[['gig_price_from']], helpp))

In [25]:
helppp.shape

(1686, 133)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
cos_sim = cosine_similarity(helppp)

In [28]:
cos_sim.shape

(1686, 1686)

In [29]:
def predict(gig_img_url_, top_n=11):
    data = cos_sim_df.reset_index()
    final_df = pd.concat([fiverr_df['gig_rating'], data], axis=1)
    gig_index = data[data['gig_img_url'] == gig_img_url_].index
    similarity = cos_sim[gig_index[0]].T
    
    sim_df = pd.DataFrame(similarity, columns=['similarity'])
    final_df = pd.concat([final_df, sim_df], axis=1)
    # You can also play around with the number
    final_df['gig_rating'] = final_df['gig_rating'][gig_index[0]] * final_df['similarity']
    final_df=final_df.drop(gig_index[0])
    final_df_sorted = final_df.sort_values(by='gig_rating', ascending=False).head(top_n)
    final_df_sorted['client_names'] = fiverr_df['client_names'][gig_index[0]]
    final_df_sorted = final_df_sorted[['gig_img_url', 'client_names', 'gig_rating']]
    
    return final_df_sorted

In [30]:
for_test_df=predict(fiverr_df['gig_img_url'][100])

In [31]:
for_model_df=fiverr_df[['gig_img_url', 'client_names', 'gig_rating']]

In [32]:
fiverr_df.shape

(1686, 10)

In [33]:
for i in range(len(fiverr_df)):
    pred_df=predict(fiverr_df['gig_img_url'][i])
    for_model_df=pd.concat([for_model_df, pred_df], axis=0)
    print('success '+str(i))

success 0
success 1
success 2
success 3
success 4
success 5
success 6
success 7
success 8
success 9
success 10
success 11
success 12
success 13
success 14
success 15
success 16
success 17
success 18
success 19
success 20
success 21
success 22
success 23
success 24
success 25
success 26
success 27
success 28
success 29
success 30
success 31
success 32
success 33
success 34
success 35
success 36
success 37
success 38
success 39
success 40
success 41
success 42
success 43
success 44
success 45
success 46
success 47
success 48
success 49
success 50
success 51
success 52
success 53
success 54
success 55
success 56
success 57
success 58
success 59
success 60
success 61
success 62
success 63
success 64
success 65
success 66
success 67
success 68
success 69
success 70
success 71
success 72
success 73
success 74
success 75
success 76
success 77
success 78
success 79
success 80
success 81
success 82
success 83
success 84
success 85
success 86
success 87
success 88
success 89
success 90
success 9

In [34]:
for_model_df.shape

(20232, 3)

In [35]:
for_model_df.to_csv('/Users/rsoedarnadi/Downloads/for_model_df.csv')

In [36]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs

In [37]:
ratings = tf.data.Dataset.from_tensor_slices(dict(for_model_df[['gig_img_url','client_names', 'gig_rating']]))
imgs = tf.data.Dataset.from_tensor_slices(dict(for_model_df[['gig_img_url']]))

ratings = ratings.map(lambda x: {
    'gig_img_url': x['gig_img_url'],
    'client_names': x['client_names'],
    'gig_rating': float(x['gig_rating'])
})

imgs = imgs.map(lambda x: x['gig_img_url'])

In [38]:
unique_gig_imgs = np.unique(np.concatenate(list(imgs.batch(1000))))
unique_client_names = np.unique(np.concatenate(list(ratings.batch(1000).map(lambda x: x["client_names"]))))
tf.random.set_seed(42)
shuffled = ratings.shuffle(20_232, seed=42, reshuffle_each_iteration=False)
train = ratings.take(16_000)
test = ratings.skip(16_000).take(4_232)

2023-06-13 20:10:25.966929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20232]
	 [[{{node Placeholder/_0}}]]
2023-06-13 20:10:26.044079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [20232]
	 [[{{node Placeholder/_2}}]]


In [40]:
embedding_dimension = 64

client_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_client_names, mask_token=None),
        tf.keras.layers.Embedding(len(unique_client_names) + 1,   
                                  embedding_dimension)
])

item_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_gig_imgs, mask_token=None),
        tf.keras.layers.Embedding(len(unique_gig_imgs) + 1, 
                                  embedding_dimension)
])

In [47]:
class Model(tfrs.Model):
  def __init__(self) -> None:
    super().__init__()
    self.item_model: tf.keras.Model = item_model
    self.client_model: tf.keras.Model = client_model
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=imgs.batch(128).map(self.item_model)
        )
    )
    
  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    client_embeddings = self.client_model(features["client_names"])
    # And pick out the movie features and pass them into the movie model.
    positive_gig_embeddings = self.item_model(features["gig_img_url"])
    
    return (
        client_embeddings,
        positive_gig_embeddings
    )
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    ratings = features.pop("gig_rating")
    client_embeddings, positive_gig_embeddings = self(features)

    retrieval_loss = self.retrieval_task(client_embeddings, positive_gig_embeddings)
    # The task computes the loss and the metrics.

    return (retrieval_loss)

In [44]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=imgs.batch(128).map(item_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)


In [48]:
cached_train = train.shuffle(20_232).batch(1000).cache()
cached_test = test.batch(1000).cache()

model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

model.fit(cached_train, epochs=10)

model.evaluate(cached_test, return_dict=True)

Epoch 1/10


2023-06-13 20:12:55.957375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [20232]
	 [[{{node Placeholder/_1}}]]
2023-06-13 20:12:58.467834: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]


 1/16 [>.............................] - ETA: 47s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0010 - factorized_top_k/top_100_categorical_accuracy: 0.0020 - loss: 6907.8296 - regularization_loss: 0.0000e+00 - total_loss: 6907.8296

2023-06-13 20:12:59.134821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_12}}]]


 2/16 [==>...........................] - ETA: 9s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0010 - factorized_top_k/top_100_categorical_accuracy: 0.0020 - loss: 6907.7932 - regularization_loss: 0.0000e+00 - total_loss: 6907.7932 

...

2023-06-13 20:15:51.897030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]


4/5 [=======================>......] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0060 - factorized_top_k/top_5_categorical_accuracy: 0.0080 - factorized_top_k/top_10_categorical_accuracy: 0.0172 - factorized_top_k/top_50_categorical_accuracy: 0.1103 - factorized_top_k/top_100_categorical_accuracy: 0.1960 - loss: 6609.0414 - regularization_loss: 0.0000e+00 - total_loss: 6609.0414

2023-06-13 20:15:52.664944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype resource
	 [[{{node Placeholder/_10}}]]


5/5 [==============================] - 5s 656ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0057 - factorized_top_k/top_5_categorical_accuracy: 0.0076 - factorized_top_k/top_10_categorical_accuracy: 0.0168 - factorized_top_k/top_50_categorical_accuracy: 0.1070 - factorized_top_k/top_100_categorical_accuracy: 0.1895 - loss: 4833.8975 - regularization_loss: 0.0000e+00 - total_loss: 4833.8975


{'factorized_top_k/top_1_categorical_accuracy': 0.005671077407896519,
 'factorized_top_k/top_5_categorical_accuracy': 0.007561436854302883,
 'factorized_top_k/top_10_categorical_accuracy': 0.01677693799138069,
 'factorized_top_k/top_50_categorical_accuracy': 0.10704158991575241,
 'factorized_top_k/top_100_categorical_accuracy': 0.18950851261615753,
 'loss': 1283.60986328125,
 'regularization_loss': 0,
 'total_loss': 1283.60986328125}

In [62]:
def predict_gigs(client, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.client_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((imgs.batch(1000), imgs.batch(1000).map(model.item_model)))
    )

    # Get recommendations.
    _, gigs = index(tf.constant([str(client)]))
    
    print('Recommendations for user {}: {}\n'.format(client, gigs[0, :3]))
   # for i, gig in enumerate(gigs[0, :top_n].numpy()):
   #     print('{}'.format(gig.decode("utf-8")))

In [63]:
predict_gigs('lifehossain')

2023-06-14 05:57:23.591076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [20232]
	 [[{{node Placeholder/_0}}]]
2023-06-14 05:57:23.591920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int64
	 [[{{node Placeholder/_13}}]]


Recommendations for user lifehossain: [b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/163754892/original/9fdc89de7b574bfcd213695eab1a76769a5deef7.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/163754892/original/9fdc89de7b574bfcd213695eab1a76769a5deef7.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/163754892/original/9fdc89de7b574bfcd213695eab1a76769a5deef7.jpg']



In [61]:
model.save_weights('tfrs.h5')